KNN - Define the distance metric between items in our data set find k closest items. we can use these items by letting then let them vote to predict some of the properties of test item.

In [17]:
import operator
import numpy as np
import pandas as pd
from scipy import spatial

column_names = ['user_id', 'movie_id', 'rating']
ratingsData = pd.read_csv( 'ml-100k/u.data', sep='\t', names = column_names, usecols = range(3) )
# ratingsData.head()

groupby movies by movie id and add average of ratings

In [18]:
moviesWithRatings = ratingsData.groupby('movie_id').agg({'rating' : [np.size, np.mean]})
# moviesWithRatings.head()

Now lets normalize the ratings from 0-1 ( nobody rated - most rated or most popular ), let's create dataframe out of movie ratings.

In [19]:
moviesWithRatingsCount = pd.DataFrame(moviesWithRatings['rating']['size'])
# moviesWithRatingsCount.head()
moviesMormalizedRatingsCounts = moviesWithRatingsCount.apply( lambda x : ( x - np.min(x) ) / (np.max(x) - np.min(x)) )
# moviesMormalizedRatingsCounts.head()

Let's create a movie dictionary which will have more then one genre which will be mapped to 1 and 0 if genre is mentioned it will be 1 else 0, we wl also add name and average rating in the dictionary

In [20]:
movieDict = {}
with open(r'ml-100k/u.item') as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map( int, genres)
        movieDict[movieID] = ( name, np.array(list(genres)), moviesMormalizedRatingsCounts.loc[movieID].get('size'), moviesWithRatings.loc[movieID].rating.get('mean') )
print(movieDict)

{1: ('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.77358490566037741, 3.8783185840707963), 2: ('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076), 3: ('Four Rooms (1995)', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.15265866209262435, 3.0333333333333332), 4: ('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.35677530017152659, 3.5502392344497609), 5: ('Copycat (1995)', array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.14579759862778729, 3.3023255813953489), 6: ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.042881646655231559, 3.5769230769230771), 7: ('Twelve Monkeys (1995)', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), 0.67066895368782165, 3.7984693877551021), 8: ('Babe (1995)', array([0, 0, 0

In [21]:
def handleComputeDistances( a, b ) :
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine( genresA, genresB )
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs( popularityA - popularityB )
    return ( genreDistance + popularityDistance )

handleComputeDistances( movieDict[2], movieDict[4] )
#Higher the distance = less simmilar movies

0.80045740423098921

In [22]:
print(movieDict[2])
print(movieDict[4])
# Genres are different

('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.35677530017152659, 3.5502392344497609)


In [35]:
def handleGetNeighbors( intMovieId, K ) :
    distances = []
    for movie in movieDict :
        if( movie != intMovieId ) :
            dist = handleComputeDistances( movieDict[intMovieId], movieDict[movie] )
            distances.append( ( movie, dist ) )
    distances.sort( key = operator.itemgetter( 1 ) )
    neighbors = []
    for x in range( K ) :
        neighbors.append( distances[x][0] )
    return neighbors

K = 10
averageRating = 0
neighbours = handleGetNeighbors( 1, K )
for intMovieId in neighbours :
    averageRating = averageRating + movieDict[intMovieId][3]
    print( movieDict[intMovieId][0] + " " + str( movieDict[intMovieId][3] ) )
averageRating = averageRating / K

averageRating

Liar Liar (1997) 3.15670103093
Aladdin (1992) 3.81278538813
Willy Wonka and the Chocolate Factory (1971) 3.63190184049
Monty Python and the Holy Grail (1974) 4.0664556962
Full Monty, The (1997) 3.92698412698
George of the Jungle (1997) 2.68518518519
Beavis and Butt-head Do America (1996) 2.78846153846
Birdcage, The (1996) 3.44368600683
Home Alone (1990) 3.08759124088
Aladdin and the King of Thieves (1996) 2.84615384615


3.3445905900235564

In [36]:
movieDict[1]
# Movie ratings of movie id comparing with results above

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.77358490566037741,
 3.8783185840707963)